In [2]:
import numpy as np
import cv2
import pandas as pd

In [3]:
brawler_by_event_map = pd.read_csv('datasets/teams/brawler_by_event_map.csv', index_col=0)
maplist = brawler_by_event_map[['event.mode','event.map']].drop_duplicates()
maplist.columns=['modo','mapa']

maplist

,modo,mapa
0,knockout,X Marks the Spot
1,knockout,Riverside
2,knockout,Out in the Open
3,knockout,New Perspective
4,knockout,Goldarm Gulch
5,knockout,Flaring Phoenix
6,knockout,Belle's Rock
7,hotZone,Split
8,hotZone,Ring of Fire
9,hotZone,Quarter Pounder


In [16]:
# contar arbustos por mapa

bushList = []

for i, mapname in enumerate(maplist['mapa']):
	# Carga la imagen donde se buscará
	img_rgb = cv2.imread("resources/maps/" + mapname + ".png", 0)

	# Carga la imagen a buscar
	img_gray = cv2.imread("resources/maps/bush_detection/grassred3.png", 0)

	# Utiliza el método matchTemplate de OpenCV para buscar la imagen en la otra
	res = cv2.matchTemplate(img_rgb, img_gray, cv2.TM_CCOEFF_NORMED)

	# Utiliza la función threshold para obtener una máscara con los puntos donde se encontró una coincidencia
	loc = np.where(res >= 0.9)

	puntos=list(zip(*loc[::-1]))

	def filtrar_puntos_por_distancia(puntos, min_dist):
		puntos_filtrados = []
		for point in puntos:
			is_valid = True
			point = np.array(point)
			for point_filtrado in puntos_filtrados:
				point_filtrado = np.array(point_filtrado)
				dist = np.linalg.norm(point - point_filtrado)
				if dist < min_dist:
					is_valid = False
					break
			if is_valid:
				puntos_filtrados.append(point)
		return puntos_filtrados

	puntos = filtrar_puntos_por_distancia(puntos, 30)

	for point in puntos:
		cv2.drawMarker(img_rgb, point, (0,0,255), cv2.MARKER_DIAMOND, 20, 2)

	# Muestra el resultado 
	# print("La imagen aparece " + str(len(puntos)) + " veces en la imagen")
	# cv2.imwrite("resources/maps/bush_detection/" + mapname + ".png", img_rgb)
	
	puntos_centro = [p for p in puntos if (p[0]>150 and p[0]<540 and p[1]<720 and p[1]>330)]
	puntos_centro

	for point in puntos_centro:
		cv2.drawMarker(img_rgb, point, (255,0,0), cv2.MARKER_CROSS, 20, 2)

	cv2.imwrite("resources/maps/bush_detection/" + mapname + ".png", img_rgb)

	bushList.append({'scope': 'total',mapname: len(puntos)})
	bushList.append({'scope': 'centro',mapname: len(puntos_centro)})

bushList

[{'scope': 'total', 'X Marks the Spot': 78},
 {'scope': 'centro', 'X Marks the Spot': 46},
 {'scope': 'total', 'Riverside': 168},
 {'scope': 'centro', 'Riverside': 14},
 {'scope': 'total', 'Out in the Open': 71},
 {'scope': 'centro', 'Out in the Open': 30},
 {'scope': 'total', 'New Perspective': 102},
 {'scope': 'centro', 'New Perspective': 2},
 {'scope': 'total', 'Goldarm Gulch': 123},
 {'scope': 'centro', 'Goldarm Gulch': 12},
 {'scope': 'total', 'Flaring Phoenix': 113},
 {'scope': 'centro', 'Flaring Phoenix': 36},
 {'scope': 'total', "Belle's Rock": 36},
 {'scope': 'centro', "Belle's Rock": 0},
 {'scope': 'total', 'Split': 94},
 {'scope': 'centro', 'Split': 14},
 {'scope': 'total', 'Ring of Fire': 114},
 {'scope': 'centro', 'Ring of Fire': 17},
 {'scope': 'total', 'Quarter Pounder': 31},
 {'scope': 'centro', 'Quarter Pounder': 4},
 {'scope': 'total', 'Parallel Plays': 108},
 {'scope': 'centro', 'Parallel Plays': 5},
 {'scope': 'total', 'Open Zone': 156},
 {'scope': 'centro', 'Open Z

In [38]:
# contar arbustos por mapa

barrierList = []

barrierTemplates = ['barriertop1.png','barriertop2.png','barriertop3.png','barriertop4.png','barriertop5.png','barriertop6.png','barriertop7.png']

for i, mapname in enumerate(maplist['mapa']):
	puntos = []
	# Carga la imagen donde se buscará
	img_rgb = cv2.imread("resources/maps/" + mapname + ".png", 0)

	for i, template in enumerate(barrierTemplates):
		# Carga la imagen a buscar
		img_gray = cv2.imread("resources/maps/barrier_detection/" + template, 0)

		# Utiliza el método matchTemplate de OpenCV para buscar la imagen en la otra
		res = cv2.matchTemplate(img_rgb, img_gray, cv2.TM_CCOEFF_NORMED)

		# Utiliza la función threshold para obtener una máscara con los puntos donde se encontró una coincidencia
		loc = np.where(res >= 0.9)

		puntos = puntos + list(zip(*loc[::-1]))

	def filtrar_puntos_por_distancia(puntos, min_dist):
		puntos_filtrados = []
		for point in puntos:
			is_valid = True
			point = np.array(point)
			for point_filtrado in puntos_filtrados:
				point_filtrado = np.array(point_filtrado)
				dist = np.linalg.norm(point - point_filtrado)
				if dist < min_dist:
					is_valid = False
					break
			if is_valid:
				puntos_filtrados.append(point)
		return puntos_filtrados

	puntos = filtrar_puntos_por_distancia(puntos, 30)

	for point in puntos:
		cv2.drawMarker(img_rgb, point, (0,0,255), cv2.MARKER_DIAMOND, 20, 2)

	# Muestra el resultado
	# print("La imagen aparece " + str(len(puntos)) + " veces en la imagen")
	# cv2.imwrite("resources/maps/bush_detection/" + mapname + ".png", img_rgb)
	
	puntos_centro = [p for p in puntos if (p[0]>150 and p[0]<540 and p[1]<720 and p[1]>330)]
	puntos_centro

	for point in puntos_centro:
		cv2.drawMarker(img_rgb, point, (255,0,0), cv2.MARKER_CROSS, 20, 2)

	cv2.imwrite("resources/maps/barrier_detection/" + mapname + ".png", img_rgb)

	barrierList.append({'scope': 'total',mapname: len(puntos)})
	barrierList.append({'scope': 'centro',mapname: len(puntos_centro)})

barrierList

[{'scope': 'total', 'X Marks the Spot': 39},
 {'scope': 'centro', 'X Marks the Spot': 15},
 {'scope': 'total', 'Riverside': 90},
 {'scope': 'centro', 'Riverside': 8},
 {'scope': 'total', 'Out in the Open': 48},
 {'scope': 'centro', 'Out in the Open': 9},
 {'scope': 'total', 'New Perspective': 52},
 {'scope': 'centro', 'New Perspective': 18},
 {'scope': 'total', 'Goldarm Gulch': 4},
 {'scope': 'centro', 'Goldarm Gulch': 4},
 {'scope': 'total', 'Flaring Phoenix': 97},
 {'scope': 'centro', 'Flaring Phoenix': 22},
 {'scope': 'total', "Belle's Rock": 102},
 {'scope': 'centro', "Belle's Rock": 24},
 {'scope': 'total', 'Split': 54},
 {'scope': 'centro', 'Split': 30},
 {'scope': 'total', 'Ring of Fire': 16},
 {'scope': 'centro', 'Ring of Fire': 0},
 {'scope': 'total', 'Quarter Pounder': 48},
 {'scope': 'centro', 'Quarter Pounder': 14},
 {'scope': 'total', 'Parallel Plays': 50},
 {'scope': 'centro', 'Parallel Plays': 0},
 {'scope': 'total', 'Open Zone': 16},
 {'scope': 'centro', 'Open Zone': 6}